Poker Bot built using TensorFlow. Trained with reinforcement learning using the PyPokerEngine library.

In [1]:
# pip install PyPokerEngine

In [4]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

"""
from tensorflow.keras import layers, losses
from tensorflow.keras.layers import Dense, Flatten, Reshape, LeakyReLU
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam, RMSprop
from collections import Counter
from datetime import datetime
import keras
import keras.callbacks
from keras.callbacks import TensorBoard
 """

In [5]:
from pypokerengine.players import BasePokerPlayer
from pypokerengine.api.emulator import Emulator
from pypokerengine.utils.game_state_utils import restore_game_state